In [4]:
%run ../../scripts/bios_utils.py

In [1]:
import escher
import modelseed_escher
from modelseed_escher.map import EscherCluster

In [2]:
escher_manager = modelseed_escher.EscherManager(escher)

In [5]:
data = read_json('/Users/fliu/workspace/jupyter/data/www/annotation/data/latest_cluster.json')
em = modelseed_escher.core.EscherMap(data['escher_map'])
em_grid = data['grid']

In [6]:
#em.display_in_notebook()

In [7]:
%run ../escher_grid_merge.py

In [8]:
em_layer = add_layers(em.clone(), em_grid, escher_manager)

150 ModelSEED.Ribitol et al Utilization 0 0 4500 2427 c


In [9]:
#em_layer.display_in_notebook()

In [10]:
ec = EscherCluster(25)
cluster_data = ec.cluster(em_layer)
result = report(cluster_data, em_layer)
result = sort_by_database(result, 'seed.compound')

In [11]:
result

{'models': ['iMM904'],
 'records': [{'model_data': {'c': {'iMM904': ['M_atp_c']}},
   'database': {'seed.compound': ['cpd00002']}},
  {'model_data': {'c': {'iMM904': ['M_fru_c']}},
   'database': {'seed.compound': ['cpd00082']}},
  {'model_data': {'c': {'iMM904': ['M_adp_c']}},
   'database': {'seed.compound': ['cpd00008']}},
  {'model_data': {'c': {'iMM904': ['M_h_c']}},
   'database': {'seed.compound': ['cpd00067']}},
  {'model_data': {'c': {'iMM904': ['M_nad_c']}},
   'database': {'seed.compound': ['cpd00003']}},
  {'model_data': {'c': {'iMM904': ['M_xylt_c']}},
   'database': {'seed.compound': ['cpd00306']}},
  {'model_data': {'c': {'iMM904': ['M_nadh_c']}},
   'database': {'seed.compound': ['cpd00004']}},
  {'model_data': {'c': {'iMM904': ['M_xylu__D_c']}},
   'database': {'seed.compound': ['cpd00259']}},
  {'model_data': {'c': {'iMM904': ['M_sbt__D_c']}},
   'database': {'seed.compound': ['cpd00588']}},
  {'model_data': {'c': {'iMM904': ['M_ru5p__D_c']}},
   'database': {'seed.co

In [54]:
builder = escher.Builder(map_json=json.dumps(em.escher_map))

In [93]:
import logging
logger = logging.getLogger(__name__)
def report(cluster_data, em):
    cluster_map = em.escher_map
    any_merge = True
    result = {
        'models' : set(),
        'records' : []
    }
    for cluster in cluster_data:
        logger.debug("%s", cluster)
        record = {
            'model_data' : {},
            'database' : {}
        }
        match_cmp = '?'
        cmps = set()
        for id in cluster:
            node_id = cluster_map[1]['nodes'][id]['bigg_id']
            logger.debug("%s", node_id)
            if node_id.startswith('cpd') and '_' in node_id:
                cpd_id, cmp = node_id.split('_')
                cmps.add(cmp)

        if len(cmps) == 1:
            match_cmp = cmps.pop()
        for id in cluster:
            node_id = cluster_map[1]['nodes'][id]['bigg_id']
            cpd_id = node_id
            database = None
            if '@' in node_id:
                cpd_id, database = node_id.split('@')

            #print(cpd_id, database)
            #if is model
            if not database == None:
                result['models'].add(database)
                cmp = match_cmp
                if not cmp in record['model_data']:
                    record['model_data'][cmp] = {}
                if not database in record['model_data'][cmp]:
                    record['model_data'][cmp][database] = set()
                record['model_data'][cmp][database].add(cpd_id)
            else:
                database = 'seed.compound'
                if not database in record['database']:
                    record['database'][database] = set()
                if node_id.startswith('cpd') and '_' in node_id:
                    cpd_id, cmp = node_id.split('_')
                record['database'][database].add(cpd_id)
            #elseif database
        if len(record['model_data']) > 0:
            result['records'].append(record)


    result['models'] = list(result['models'])
    for r in result['records']:
        for cmp_id in r['model_data']:
            for model_id in r['model_data'][cmp_id]:
                r['model_data'][cmp_id][model_id] = list(r['model_data'][cmp_id][model_id])
        for database_id in r['database']:
            r['database'][database_id] = list(r['database'][database_id])
            
    return result
dd = report(cluster_data, em_layer)



In [94]:
dd['records'][0]

{'model_data': {'c': {'iMM904': ['M_nadp_c']}},
 'database': {'seed.compound': ['cpd00006']}}

In [98]:
def sort_by_database(report, db):
    result = {
        'models' : report['models']
    }
    m = {}
    records = []
    for r in report['records']:
        if 'seed.compound' in r['database']:
            key = tuple(sorted(r['database']['seed.compound']))
            if not key in m:
                m[key] = {}
            for cmp in r['model_data']:
                if not cmp in m[key]:
                    m[key][cmp] = {}
                for model_id in r['model_data'][cmp]:
                    if not model_id in m[key][cmp]:
                        m[key][cmp][model_id] = set()
                    m[key][cmp][model_id] |= set(r['model_data'][cmp][model_id])
        else:
            records.append(r)
    for key in m:
        record = {
            'model_data' : {},
            'database' : {db : list(key)}
        }
        for cmp in m[key]:
            record[cmp] = {}
            for model_id in m[key][cmp]:
                record[cmp][model_id] = list(m[key][cmp][model_id])
        records.append(record)
        print(record)
    result['records'] = records
    return report
    
sort_by_database(dd, 'seed.compound')


{'model_data': {}, 'database': {'seed.compound': ['cpd00006']}, 'c': {'iMM904': ['M_nadp_c'], 'iAL1006': ['M_NADP']}, 'm': {'iMM904': ['M_nadp_m'], 'iAL1006': ['M_NADPm']}}
{'model_data': {}, 'database': {'seed.compound': ['cpd00005']}, 'c': {'iMM904': ['M_nadph_c'], 'iAL1006': ['M_NADPH']}, 'm': {'iMM904': ['M_nadph_m'], 'iAL1006': ['M_NADPHm']}}
{'model_data': {}, 'database': {'seed.compound': ['cpd00067']}, 'c': {'iMM904': ['M_h_c'], 'iAL1006': ['M_H_PO']}, 'm': {'iMM904': ['M_h_m'], 'iAL1006': ['M_H_POm']}}
{'model_data': {}, 'database': {'seed.compound': ['cpd00330']}, 'c': {'iMM904': ['M_dhf_c'], 'iAL1006': ['M_DHF']}, 'm': {'iMM904': ['M_dhf_m'], 'iAL1006': ['M_DHFm']}}
{'model_data': {}, 'database': {'seed.compound': ['cpd00001']}, 'c': {'iMM904': ['M_h2o_c'], 'iAL1006': ['M_H2O']}, 'm': {'iMM904': ['M_h2o_m'], 'iAL1006': ['M_H2Om']}}
{'model_data': {}, 'database': {'seed.compound': ['cpd00038']}, 'c': {'iMM904': ['M_gtp_c'], 'iAL1006': ['M_GTP']}, 'm': {'iMM904': ['M_gtp_m'], 

{'models': ['iMM904', 'iAL1006'],
 'records': [{'model_data': {'c': {'iMM904': ['M_nadp_c']}},
   'database': {'seed.compound': ['cpd00006']}},
  {'model_data': {'c': {'iMM904': ['M_nadph_c']}},
   'database': {'seed.compound': ['cpd00005']}},
  {'model_data': {'c': {'iMM904': ['M_h_c']}},
   'database': {'seed.compound': ['cpd00067']}},
  {'model_data': {'c': {'iMM904': ['M_nadp_c']}},
   'database': {'seed.compound': ['cpd00006']}},
  {'model_data': {'c': {'iMM904': ['M_nadph_c']}},
   'database': {'seed.compound': ['cpd00005']}},
  {'model_data': {'c': {'iMM904': ['M_h_c']}},
   'database': {'seed.compound': ['cpd00067']}},
  {'model_data': {'c': {'iMM904': ['M_dhf_c']}},
   'database': {'seed.compound': ['cpd00330']}},
  {'model_data': {'c': {'iMM904': ['M_h2o_c']}},
   'database': {'seed.compound': ['cpd00001']}},
  {'model_data': {'c': {'iMM904': ['M_gtp_c']}},
   'database': {'seed.compound': ['cpd00038']}},
  {'model_data': {'c': {'iMM904': ['M_h_c']}},
   'database': {'seed.co

In [95]:
dd

{'models': ['iMM904', 'iAL1006'],
 'records': [{'model_data': {'c': {'iMM904': ['M_nadp_c']}},
   'database': {'seed.compound': ['cpd00006']}},
  {'model_data': {'c': {'iMM904': ['M_nadph_c']}},
   'database': {'seed.compound': ['cpd00005']}},
  {'model_data': {'c': {'iMM904': ['M_h_c']}},
   'database': {'seed.compound': ['cpd00067']}},
  {'model_data': {'c': {'iMM904': ['M_nadp_c']}},
   'database': {'seed.compound': ['cpd00006']}},
  {'model_data': {'c': {'iMM904': ['M_nadph_c']}},
   'database': {'seed.compound': ['cpd00005']}},
  {'model_data': {'c': {'iMM904': ['M_h_c']}},
   'database': {'seed.compound': ['cpd00067']}},
  {'model_data': {'c': {'iMM904': ['M_dhf_c']}},
   'database': {'seed.compound': ['cpd00330']}},
  {'model_data': {'c': {'iMM904': ['M_h2o_c']}},
   'database': {'seed.compound': ['cpd00001']}},
  {'model_data': {'c': {'iMM904': ['M_gtp_c']}},
   'database': {'seed.compound': ['cpd00038']}},
  {'model_data': {'c': {'iMM904': ['M_h_c']}},
   'database': {'seed.co